In [22]:
#set environment
import os
import sys
 
os.environ["SPARK_HOME"] = "/usr/hdp/current/spark2-client"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/local/anaconda/bin/python" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/local/anaconda/bin/python"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.4-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

In [23]:
#import Sparksession driver
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Classification of Student dataset") \
    .getOrCreate()

In [25]:
student = spark.read.csv('data/student_copy.csv',header=True,inferSchema=True)
student.show(5)

+-----+-----+--------+----------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|instr|class|nbrepeat|attendance|difficulty| Q1| Q2| Q3| Q4| Q5| Q6| Q7| Q8| Q9|Q10|Q11|Q12|Q13|Q14|Q15|Q16|Q17|Q18|Q19|Q20|Q21|Q22|Q23|Q24|Q25|Q26|Q27|Q28|
+-----+-----+--------+----------+----------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
|    1|    2|       1|         0|         4|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|
|    1|    2|       1|         1|         3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|  3|
|    1|    2|       1|         2|         4|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|  5|
|    1|    2|       1|         1|         3|  3|  3|  3|  

# MISSING VALUES

In [26]:
#Check for missing values
for col in student.columns:
    print("no. of cells in column", col, "with null values:", student.filter(student[col].isNull()).count())

no. of cells in column instr with null values: 0
no. of cells in column class with null values: 0
no. of cells in column nbrepeat with null values: 0
no. of cells in column attendance with null values: 0
no. of cells in column difficulty with null values: 0
no. of cells in column Q1 with null values: 0
no. of cells in column Q2 with null values: 0
no. of cells in column Q3 with null values: 0
no. of cells in column Q4 with null values: 0
no. of cells in column Q5 with null values: 0
no. of cells in column Q6 with null values: 0
no. of cells in column Q7 with null values: 0
no. of cells in column Q8 with null values: 0
no. of cells in column Q9 with null values: 0
no. of cells in column Q10 with null values: 0
no. of cells in column Q11 with null values: 0
no. of cells in column Q12 with null values: 0
no. of cells in column Q13 with null values: 0
no. of cells in column Q14 with null values: 0
no. of cells in column Q15 with null values: 0
no. of cells in column Q16 with null values: 0

## COPYING THE COLUMNS EXCLUSING TARGET COLUMNS

In [27]:
dfs=student.drop('instr')
dfs_colnames = dfs.columns

In [28]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
assembler = VectorAssembler(inputCols= dfs_colnames, 
                            outputCol="features")
feature_vec=assembler.transform(student)
feature_vec.select("features").take(3)

[Row(features=DenseVector([2.0, 1.0, 0.0, 4.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])),
 Row(features=DenseVector([2.0, 1.0, 1.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])),
 Row(features=DenseVector([2.0, 1.0, 2.0, 4.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0]))]

In [29]:
#Count of target classes
feature_vec.groupBy('instr').count().show()
#there is data imbalance

+-----+-----+
|instr|count|
+-----+-----+
|    1|  775|
|    3| 3601|
|    2| 1444|
+-----+-----+



## CHANGEDTHETYPE OF COLUMN INSTR TO DOUBLE ,so THAT I COULD CHECK METRIC AS CONFUSION METRIC

In [30]:
from pyspark.sql.types import DoubleType

changedTypedf = feature_vec.withColumn("instr", feature_vec["instr"].cast(DoubleType()))

In [31]:
# Split the data into train and test sets
train_data, test_data = changedTypedf.randomSplit([.75,.25],seed=0)

## LOGISTIC REGRESSION

In [32]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="instr", featuresCol="features",  
                        maxIter=100, regParam=0.0001, family="multinomial",  
                        elasticNetParam=0.0)

# Train model with Training Data
lrModel = lr.fit(train_data)
predictions = lrModel.transform(test_data)
predictions.printSchema()

root
 |-- instr: double (nullable = true)
 |-- class: integer (nullable = true)
 |-- nbrepeat: integer (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- difficulty: integer (nullable = true)
 |-- Q1: integer (nullable = true)
 |-- Q2: integer (nullable = true)
 |-- Q3: integer (nullable = true)
 |-- Q4: integer (nullable = true)
 |-- Q5: integer (nullable = true)
 |-- Q6: integer (nullable = true)
 |-- Q7: integer (nullable = true)
 |-- Q8: integer (nullable = true)
 |-- Q9: integer (nullable = true)
 |-- Q10: integer (nullable = true)
 |-- Q11: integer (nullable = true)
 |-- Q12: integer (nullable = true)
 |-- Q13: integer (nullable = true)
 |-- Q14: integer (nullable = true)
 |-- Q15: integer (nullable = true)
 |-- Q16: integer (nullable = true)
 |-- Q17: integer (nullable = true)
 |-- Q18: integer (nullable = true)
 |-- Q19: integer (nullable = true)
 |-- Q20: integer (nullable = true)
 |-- Q21: integer (nullable = true)
 |-- Q22: integer (nullable = true)
 |-- Q23: 

In [33]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='instr', metricName='accuracy')
evaluator.evaluate(predictions)

0.6265734265734266

In [34]:
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='instr', metricName='f1')
evaluator.evaluate(predictions)

0.5181398783178148

## CHECKING METRICS

In [35]:
# Make predicitons
predictionAndTarget = lrModel.transform(test_data).select("instr", "prediction")
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='instr')
# Get metrics
acc = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "f1"})
weightedPrecision = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "weightedPrecision"})
weightedRecall = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

In [36]:
print(acc)
print(f1)
print(weightedPrecision)
print(weightedRecall)

0.6265734265734266
0.5181398783178148
0.5599402778030851
0.6265734265734266


In [37]:
from pyspark.mllib.evaluation import MulticlassMetrics
import pandas as pd

predictionAndLabels = lrModel.transform(test_data).select('instr', 'prediction')
metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics.confusionMatrix().toArray()
labels = [int(l) for l in metrics.call('labels')]
confusion_matrix = pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [38]:
confusion_matrix

,1,2,3
1,9.0,3.0,8.0
2,10.0,22.0,20.0
3,156.0,337.0,865.0


### PARAMETER TUNNING

In [39]:
#Grid Search
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

paramGrid = (ParamGridBuilder()\
             .addGrid(lr.regParam,[0.001,0.01,0.1,1])\
             .addGrid(lr.elasticNetParam,[0.0,0.5,1.0])\
             .build())

# Create 4-fold CrossValidator
cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=4)

cvModel = cv.fit(train_data)

In [40]:
#Best Model Params
score_params_list = list(zip(cvModel.avgMetrics, cvModel.getEstimatorParamMaps()))
max(score_params_list,key=lambda item:item[0])

(0.4993116485249862,
 {Param(parent='LogisticRegression_424d811739e639f5be2b', name='regParam', doc='regularization parameter (>= 0).'): 0.001,
  Param(parent='LogisticRegression_424d811739e639f5be2b', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0})

In [21]:
predictions = cvModel.bestModel.transform(test_data)
evaluator.evaluate(predictions)

0.5181332770507456

In [22]:
# Make predicitons
predictionAndTarget = cvModel.transform(test_data).select("instr", "prediction")
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='instr')
# Get metrics
acc = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "f1"})
weightedPrecision = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "weightedPrecision"})
weightedRecall = evaluator.evaluate(predictionAndTarget, {evaluator.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget)

In [23]:
print(acc)
print(f1)
print(weightedPrecision)
print(weightedRecall)

0.6265734265734266
0.5181332770507456
0.5608179292937531
0.6265734265734266


In [35]:
import pandas as pd
predictionAndLabels = cvModel.transform(test_data).select('instr', 'prediction')
metrics = MulticlassMetrics(predictionAndLabels.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics.confusionMatrix().toArray()
labels = [int(l) for l in metrics.call('labels')]
confusion_matrix = pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [36]:
confusion_matrix

,1,2,3
1,9.0,3.0,7.0
2,10.0,22.0,21.0
3,156.0,337.0,865.0


# LOGISTIC REGRESSION on SKLEARN gave ACCURACY SCORE 61% ,in PYSPARK RESULATED in SAME SCORE 62% EVEN AFTER TUNNING 

## WITH PCA 

## SCALED THE FEATURES

In [41]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)
scalerModel = scaler.fit(train_data)
scaledData = scalerModel.transform(train_data)
scaledData_test = scalerModel.transform(test_data)
scaledData.select("scaledFeatures").take(3)

[Row(scaledFeatures=DenseVector([-1.4304, -0.3958, -1.1209, -1.3124, -1.4362, -1.6111, -1.736, -1.6195, -1.6454, -1.6391, -1.6099, -1.5935, -1.7075, -1.6382, -1.6961, -1.5663, -1.7696, -1.8233, -1.8179, -1.6803, -1.888, -1.7328, -1.7766, -1.7914, -1.8132, -1.8328, -1.7278, -1.6986, -1.8328, -1.7482, -1.671, -1.7975])),
 Row(scaledFeatures=DenseVector([-1.4304, -0.3958, -1.1209, -1.3124, -1.4362, -1.6111, -1.736, -1.6195, -1.6454, -1.6391, -1.6099, -1.5935, -1.7075, -1.6382, -1.6961, -1.5663, -1.7696, -1.8233, -1.8179, -1.6803, -1.888, -1.7328, -1.7766, -1.7914, -1.8132, -1.8328, -1.7278, -1.6986, -1.8328, -1.7482, -1.671, -1.7975])),
 Row(scaledFeatures=DenseVector([-1.4304, -0.3958, -1.1209, -1.3124, 0.0571, -0.0546, -1.736, -0.0602, -1.6454, -0.8577, -0.8276, -0.8115, -0.9191, -1.6382, -1.6961, -1.5663, -0.1853, 0.5727, 0.5763, -0.1251, -1.0978, 0.617, -1.7766, -0.2209, -0.2371, -0.2489, -0.9412, -0.1272, 0.5583, -0.9582, -0.119, -0.2338]))]

## APPLIED PCA

In [42]:
from pyspark.ml.feature import PCA

pca = PCA(k=5, inputCol="scaledFeatures", outputCol="pca_features")
pcamodel = pca.fit(scaledData)
pcatraindata=pcamodel.transform(scaledData)
pca_test = pcamodel.transform(scaledData_test)

In [43]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr1 = LogisticRegression(labelCol="instr", featuresCol="scaledFeatures",  
                        maxIter=100, regParam=0.0001, family="multinomial",  
                        elasticNetParam=0.0)

# Train model with Training Data
lrModel1 = lr1.fit(pcatraindata)

In [44]:
predpca = lrModel1.transform(pca_test)
predpca.printSchema()

root
 |-- instr: double (nullable = true)
 |-- class: integer (nullable = true)
 |-- nbrepeat: integer (nullable = true)
 |-- attendance: integer (nullable = true)
 |-- difficulty: integer (nullable = true)
 |-- Q1: integer (nullable = true)
 |-- Q2: integer (nullable = true)
 |-- Q3: integer (nullable = true)
 |-- Q4: integer (nullable = true)
 |-- Q5: integer (nullable = true)
 |-- Q6: integer (nullable = true)
 |-- Q7: integer (nullable = true)
 |-- Q8: integer (nullable = true)
 |-- Q9: integer (nullable = true)
 |-- Q10: integer (nullable = true)
 |-- Q11: integer (nullable = true)
 |-- Q12: integer (nullable = true)
 |-- Q13: integer (nullable = true)
 |-- Q14: integer (nullable = true)
 |-- Q15: integer (nullable = true)
 |-- Q16: integer (nullable = true)
 |-- Q17: integer (nullable = true)
 |-- Q18: integer (nullable = true)
 |-- Q19: integer (nullable = true)
 |-- Q20: integer (nullable = true)
 |-- Q21: integer (nullable = true)
 |-- Q22: integer (nullable = true)
 |-- Q23: 

## METRICS

In [45]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Make predicitons
predictionAndTarget1 = lrModel1.transform(pca_test).select("instr", "prediction")
evaluator = MulticlassClassificationEvaluator(predictionCol='prediction', labelCol='instr')
# Get metrics
acc = evaluator.evaluate(predictionAndTarget1, {evaluator.metricName: "accuracy"})
f1 = evaluator.evaluate(predictionAndTarget1, {evaluator.metricName: "f1"})
weightedPrecision = evaluator.evaluate(predictionAndTarget1, {evaluator.metricName: "weightedPrecision"})
weightedRecall = evaluator.evaluate(predictionAndTarget1, {evaluator.metricName: "weightedRecall"})
auc = evaluator.evaluate(predictionAndTarget1)

In [46]:
print(acc)
print(f1)
print(weightedPrecision)
print(weightedRecall)

0.627972027972028
0.5192429625673614
0.5617376281515962
0.627972027972028


In [47]:
import pandas as pd
from pyspark.mllib.evaluation import MulticlassMetrics

predictionAndLabels1 = lrModel1.transform(pca_test).select('instr', 'prediction')
metrics1 = MulticlassMetrics(predictionAndLabels1.rdd.map(lambda x: tuple(map(float, x))))

confusion_matrix = metrics1.confusionMatrix().toArray()
labels = [int(l) for l in metrics1.call('labels')]
confusion_matrix = pd.DataFrame(confusion_matrix , index=labels, columns=labels)

In [48]:
confusion_matrix

,1,2,3
1,9.0,4.0,6.0
2,11.0,22.0,20.0
3,155.0,336.0,867.0


# NOTE WITH PCA instr3 has one value increased in the TP ,one decrease each in FP FOR inst1 and isnt2

In [ ]:
# LOGISTIC REGRESSION with PCA on SKLEARN gave ACCURACY SCORE 61% ,in PYSPARK with PCA RESULATED in SAME SCORE 62% EVEN AFTER TUNNING 